<a href="https://colab.research.google.com/github/Hongbi-Kim/Writing_lyrics/blob/master/%EC%83%9D%EC%84%B1_%EA%B0%80%EC%82%AC_%EC%88%98%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 8.6 MB/s 
     |████████████████████████████████| 895 kB 55.8 MB/s 
     |████████████████████████████████| 596 kB 61.3 MB/s 
     |████████████████████████████████| 6.6 MB 52.6 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


- gpt3_1 : tokenizer 250, 300으로 학습
- gpt3_2 : tokenizer 350으로 학습
- gpt3_5000sample : 5000개 데이터, tokenizer 350으로 학습

In [2]:
!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git 
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab190912.sh
%cd ../

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 69.0 MB/s 
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 25.00 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-04-26 08:01:39--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::22cd:e0db, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loc

In [3]:
from konlpy.tag import Mecab
mecab = Mecab()

Exception: ignored

In [4]:
from konlpy.tag import Okt
okt = Okt()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# 구글 드라이브에서 모델, 토크나이저 불러오기
import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random
import re

from collections import Counter

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

# gpt3-kor-small_based_on_gpt2
from transformers import BertTokenizerFast, GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/5조_파이널PJT/코드/model_result/gpt3_topic')
tokenizer_gpt3 = BertTokenizerFast.from_pretrained('/content/drive/MyDrive/5조_파이널PJT/코드/model_result/gpt3_topic')
input_ids = tokenizer_gpt3.encode("text to tokenize")[1:]  # remove cls token

In [7]:
device = torch.device("cuda")
model.cuda()

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
model = model.to(device)

In [8]:
def lyric_generator(gen, lyric):
  prompt = f"<|startoftext|> <{gen}> {lyric}"
  generated = torch.tensor(tokenizer_gpt3.encode(prompt)[1:]).unsqueeze(0)
  generated = generated.to(device)

  print(generated)

  sample_outputs = model.generate(
                                  generated, 
                                  #bos_token_id=random.randint(1,30000),
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length = 300,
                                  top_p=0.95, 
                                  num_return_sequences=1,
                                  repetition_penalty=1.1
                                  )

  for i, sample_output in enumerate(sample_outputs):
    #result = "{}: {}\n\n".format(i, tokenizer_gpt3.decode(sample_output, skip_special_tokens=True))
    result = tokenizer_gpt3.decode(sample_output, skip_special_tokens=True)
    result = re.sub(r"<br>", "\n", result)
    #print(result)
  return result

In [9]:
def decide_topic(gen, lyric):

  d_topic1 = ['밤','심장','순간','날','눈','몸','숨','눈빛','머리','시선','맘','끝','손','기분','춤','시작','입술','원','준비','소리',
              '음악','느낌','때','완벽','안','위','필요','리듬','입','전']
  d_topic2 = ['눈','밤','속','순간','꿈','손','하늘','날','위','끝','숨','세상','맘','심장','시간','빛','바람','눈빛','안','기분','소리',
              '시작','별','길','때','몸','곳','노래','향기','시선']
  d_topic3 = ['남자','집','여자','매력','친구','옷','머리','생일','기분','전화','축하','노래','화장','밥','티','커피','눈치','정신','아침',
              '스타일','오늘','애','문자','몸매','분위기','얼굴','영화','오빠','번호','생일날']
  d_topic4 = ['엄마','돈','아빠','인생','나이','공부','아저씨','집','왕','애','학교','금','배','물','개','아이','맛','키','형','밥','산타',
              '오빠','몫','랩','산','동생','부모','술','라면','남']
  d_topic5 = ['춤','몸','음악','머리','인생','볼륨','준비','리듬','소리','다리','무대','돈','잔','비트','걱정','남','승리','젋음','발','폼',
              '모두','허리','땀','함성','해변','엉덩이','차','피해','게임','해결']

  b_topic1 = ['꿈','속','하늘','빛','위','끝','세상','순간','길','별','손','어둠','숨','곳','시간','시작','바람','안','눈','밤','미래','태양',
              '현실','구름','아래','날','달','내일','달빛','삶']
  b_topic2 = ['사랑','말','날','마음','생각','맘','사람','시간','모습','속','때','가슴','행복','곁','눈물','기억','세상','하루','이상','일','눈',
              '오늘','앞','끝','여자','필요','추억','친구','혼자','밤']
  b_topic3 = ['돈','집','엄마','친구','인생','나이','밥','아빠','술','오빠','차','키','남','여자','부모','잔','학교','동네','아저씨','크리스마스',
              '라면','한잔','결혼','공부','우정','애','모두','언니','배','누나']
  b_topic4 = ['춤','음악','바다','리듬','위','파도','여름','노래','몸','소리','밤','아래','태양','바람','하늘','조명','도시','볼륨','햇살','분위기',
              '기분','여행','모래','파티','걱정','일상','불빛','무대','산','해']
  b_topic5 = ['눈','맘','날','눈빛','밤','기분','심장','머리','손','순간','말','입술','시선','향기','몸','느낌','입','때','원','숨','상상','남자',
              '눈치','매력','준비','완벽','오늘','전','미소','표정']

  token_list = []
  for i in lyric:
    #token = mecab.pos(i)
    token = okt.pos(i)
    if token != []:
      token_list.append(token)
  #print(token_list)

  word_list = []
  for token in token_list:
    #if token[0][1] == 'NNG':
    if token[0][1] == 'Noun':
      word_list.append(token[0][0])
  #print(word_list)
  
  topic_counts = []
  # Counter().moset_common(n)의 경우 같은 수의 카운트는 발생한 순으로 반환
  # 댄스 1-2-3-4-5
  if gen == '댄스':
    for word in word_list:
      if word in d_topic1:
        topic_counts.append('topic1')
      if word in d_topic2:
        topic_counts.append('topic2')
      if word in d_topic3:
        topic_counts.append('topic3')
      if word in d_topic4:
        topic_counts.append('topic4')
      if word in d_topic5:
        topic_counts.append('topic5')

  # 발라드 1-2-3-5-4
  elif gen == '발라드':
    for word in word_list:
      if word in b_topic1:
        topic_counts.append('topic1')
      if word in b_topic2:
        topic_counts.append('topic2')
      if word in b_topic3:
        topic_counts.append('topic3')
      if word in b_topic5:
        topic_counts.append('topic5')
      if word in b_topic4:
        topic_counts.append('topic4')  

  if topic_counts == []:
    topic = 'topic6'
  else:
    topic = Counter(topic_counts).most_common(1)[0][0]
  
  print(topic_counts)
  print(Counter(topic_counts).most_common())
  gen = gen + topic  

  return gen, lyric

In [10]:
decide_topic('댄스', '꿈 하늘 세상 엄마 아빠 춤 음악 노래')

['topic2', 'topic1', 'topic5']
[('topic2', 1), ('topic1', 1), ('topic5', 1)]


('댄스topic2', '꿈 하늘 세상 엄마 아빠 춤 음악 노래')

In [16]:
def decide_regen(input):

  val = [] # 한국어가 있으면 True, 한국어가 없으면 False
  for i in input.split('\n'):
    val.append(str(bool(re.search(r'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]',i))))

  eng = []
  space = []
  for char in input:
    if re.sub(r'[a-z]','',char) == "":
      eng += char
    elif char == " ":
      space += char
  eng_ratio = len(eng)/(len(input)-len(space))

  # 한국어가 하나도 없으면
  if (len(set(val[1:])) == 1) & ('False' in set(val[1:])) == True:
    #print((len(set(val[1:])) == 1) & ('False' in set(val[1:])))
    print(input)
    output = 'regenerate'
  # 영어의 비율이 0.7 보다 크면
  elif eng_ratio > 0.7:
    print(eng_ratio)
    print(input)
    output = 'regenerate'
  else:
    print(eng_ratio)
    output = 'pass'
  
  return output

In [17]:
def remove_lastEng(a):
  # 마지막 줄에 영어만 있다면
  if re.sub(r'[a-z\n]','',a.split('<br>')[-1]).strip() == "" :
    b = a.split('<br>')[:-1]
    c = ""
    for i in b:
      if i == ' ':
        i = re.sub(r" ", "\n \n", i)
      c += i
      c = re.sub(r"  ", "\n", c)
      output = c
  else:
    output = a.replace('<br>','\n')
  return output

In [76]:
def ai_lyrics(gen, lyric):
  #print(decide_topic(gen,lyric)[0])
  result = lyric_generator(decide_topic(gen,lyric)[0],decide_topic(gen,lyric)[1])

  if len(result) < 350:
    print('길이 재생성이 필요합니다')
    result = lyric_generator(decide_topic(gen,lyric)[0],decide_topic(gen,lyric)[1])

  if decide_regen(result) == 'regenerate':
    print('한글 재생성이 필요합니다.')
    #print(gen)
    output = ai_lyrics(gen, lyric)
  else:
    output = remove_lastEng(result)

  print(output)
  return

In [35]:
ai_lyrics('댄스','반복되는 하루 지긋지긋해')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


[]
[]
[]
[]
tensor([[42000, 42014, 16366, 14106, 14298,  6266,  9110,  8038,  9110,  8010,
             3]], device='cuda:0')


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


길이 재생성이 필요합니다
[]
[]
tensor([[42000, 42014, 16366, 14106, 14298,  6266,  9110,  8038,  9110,  8010,
             3]], device='cuda:0')
0.0
반복되는 하루 지긋지긋해 
 아 오늘이 오늘의 소중한 이 밤


In [73]:
ai_lyrics('댄스','반복되는 하루 지긋지긋해')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


[]
[]
[]
[]
tensor([[42000, 42014, 16366, 14106, 14298,  6266,  9110,  8038,  9110,  8010,
             3]], device='cuda:0')
길이 재생성이 필요합니다


NameError: ignored

In [66]:
ai_lyrics('댄스','반복되는 하루 지긋지긋해')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


[]
[]
[]
[]
tensor([[42000, 42014, 16366, 14106, 14298,  6266,  9110,  8038,  9110,  8010,
             3]], device='cuda:0')


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


길이 재생성이 필요합니다
[]
[]
[]
[]
tensor([[42000, 42014, 16366, 14106, 14298,  6266,  9110,  8038,  9110,  8010,
             3]], device='cuda:0')
0.0
반복되는 하루 지긋지긋해 
 이미 생긴 음악 
 이 음악은 모두 좋아노래 부르기 좋아 
 나의 얘긴 그냥 아는 사람줬으면 해 
 좋은 노래들은 계속 흘러나와서 계속 흘러나오기 싫어 
 이 노래가 좋다고 하는 
 나도 모르게 너무 이 났지만 
 이 노랠 들으면서 이 노랠 듣고 있는 
 나도 정말 너무 좋아 행복해 
 이 소리가 좋다고 하는 
 나도 나도 너무 너무 좋아 행복해 
 어떤 음악이 좋다고 하던 
 어떤 음악이라고 하던 
 우리만 모르는 얘기야 
 내가 누군 지도 모르는 
 어디서 본 듯한 
 그런 신나는 노래가 있는데 
 자꾸 기분이 나빠져 이 노랠듣고 있는 
 나도 정말 이 났지만 
 이 노랠 들으면서 이 노랠 듣고 있는 
 나도 정말 너무 좋아 행복해 
 이게 좋은지도 모르는데 
 좋아하면 언젠가는 언젠가는 
 언젠가는 헤어질 것 같아
